In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm

In [ ]:
import os
import jsonlines
from tqdm import tqdm

save_names = ['llava_pretrain']
data = {name: [] for name in save_names}
file_path = '/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/alignment_2.5m.jsonl'
base_path = '/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment'

# Count the total number of lines for the progress bar
total_lines = sum(1 for _ in open(file_path))

with jsonlines.open(file_path) as reader:
    for entry in tqdm(reader, total=total_lines, desc="Processing entries"):
        data_split = entry['image'].split('/')[1]
        
        if data_split in save_names:
            if data_split == 'llava_pretrain':
                entry['image'] = entry['image'].replace('llava_pretrain/images', 'sbu558k')
            image_path = os.path.join(base_path, entry['image'])
            image_path = os.path.normpath(image_path)
            
            if os.path.exists(image_path):
                data[data_split].append(entry)
            else:
                break
# count the number of each dataset
for key, entries in data.items():
    print(f"{key}: {len(entries)}")


/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/sbu558k/00120/001200924.jpg

In [ ]:
# save as json for each data
for key, entries in data.items():
    save_path = f'/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/{key}.json'
    with open(save_path, 'w') as f:
        json.dump(entries, f)
    print(f"Saved {len(entries)} entries to {save_path}")

In [ ]:
import torch
from tqdm import tqdm
def get_linear_input(sequence_output):
    cls_token = sequence_output[:, 0]
    patch_tokens = sequence_output[:, 1:]
    linear_input = torch.cat([cls_token, patch_tokens.mean(dim=1)], dim=1)
    return linear_input
directory = '/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-base'
for filename in os.listdir(directory):
    if filename.endswith(".pt"):
        print(os.path.join(directory, filename))
        image_embedding = torch.load(os.path.join(directory, filename))
        image_embedding = get_linear_input(image_embedding)
        torch.save(image_embedding, os.path.join(directory, filename))
    else:
        continue

In [ ]:
import torch
import random

def compare_random_tensor_value(pt_file1, pt_file2, position=None):
    # 加载 .pt 文件
    tensor1 = torch.load(pt_file1,weights_only=True)
    tensor2 = torch.load(pt_file2,weights_only=True)
    # 如果没有指定位置，随机选择一个索引位置
    if position is None:
        position = random.randint(0, len(tensor1) - 1) 
    # 获取指定位置的值
    value1 = tensor1[position]
    value2 = tensor2[position]
    
    # 比较两个值并打印结果
 
    print(f"The values at position {position} of index {index} are different.\n"
              f"File 1 value: {value1}\nFile 2 value: {value2}")
    
    # return value1, value2

index = random.randint(150,200)
pt_file1 = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/dreamclipcc3m_raw_caption/{index}.pt"
pt_file2 = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/dreamclipcc3m_raw/{index}.pt"
compare_random_tensor_value(pt_file1,pt_file2)

In [ ]:
random.randint(120)

In [ ]:
import glob
import os
from natsort import natsorted
path_list = ['/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/ALLaVAVFLAN', '/home/mila/l/le.zhang/scratch/light_align/data/image_embedding/dinov2-large/LLaVA558K']
all_files = []
for dir_path in path_list:
    files = glob.glob(os.path.join(dir_path, "*.pt"))
    sorted_files = natsorted(files)
    all_files.extend(sorted_files)
all_files

In [ ]:
import os
import json
# read json
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/llava_pretrain.json', 'r') as f:
    sharegpt4v = json.load(f)
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/sbu558k.json', 'r') as f:
    sbu558k = json.load(f)
with open('/home/mila/l/le.zhang/scratch/datasets/Cambrian-Alignment/jsons/coco.json', 'r') as f:
    coco = json.load(f)




In [ ]:
# function to caculate avg length 
def avg_length(data):
    avg = 0
    for entry in data:
        avg += len(entry['conversations'][-1]['value'])
    return avg/len(data)

avg_length(sharegpt4v), avg_length(sbu558k), avg_length(coco)

In [ ]:
sharegpt4v[0]

In [ ]:
import os
import torch
from natsort import natsorted
import glob

def merge_pt_files(directory):
    # 获取所有的 .pt 文件路径，并按名字顺序排序
    files = glob.glob(os.path.join(directory, "*.pt"))
    files = natsorted(files)
    
    # 确保文件数量为偶数
    if len(files) % 2 != 0:
        raise ValueError("The number of .pt files is not even. Please make sure the files can be paired.")

    # 遍历所有文件，按两个为一组进行合并
    for i in range(0, len(files), 2):
        file1 = files[i]
        file2 = files[i+1]
        
        # 加载两个文件
        tensor1 = torch.load(file1)
        tensor2 = torch.load(file2)
        
        # 在第一个维度进行拼接
        merged_tensor = torch.cat((tensor1, tensor2), dim=0)
        
        # 生成新文件名
        base_name1 = os.path.basename(file1).replace(".pt", "")
        base_name2 = os.path.basename(file2).replace(".pt", "")
        new_file_name = f"{base_name1}_{base_name2}.pt"
        new_file_path = os.path.join(directory, new_file_name)
        
        # 保存拼接后的tensor
        torch.save(merged_tensor, new_file_path)
        print(f"Saved merged tensor to {new_file_path}")

# 示例用法
# merge_pt_files('/path/to/your/directory')


In [ ]:
import os
import glob
import torch
from tqdm import tqdm
file_dir = "/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/laion30m_caption"
files = glob.glob(os.path.join(file_dir, "*.pt"))
total_len = 0
for file in tqdm(files):
    tensor = torch.load(file, map_location=torch.device('cpu'), weights_only=False)
    total_len += tensor.size(0)
    if tensor.size(0) != 4096:
        print(file, tensor.size(0))
print(total_len)



In [1]:
import os
import glob
import torch
from tqdm import tqdm
file_dir = "/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/dreamclipcc12mhf_raw_caption"
files = glob.glob(os.path.join(file_dir, "*.pt"))
total_len = 0
for file in tqdm(files):
    tensor = torch.load(file, map_location=torch.device('cpu'), weights_only=False)
    total_len += tensor.size(0)
    if tensor.size(0) != 4096:
        print(file, tensor.size(0))
print(total_len)



100%|██████████| 1880/1880 [03:54<00:00,  8.00it/s]

/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/dreamclipcc12mhf_raw_caption/1879.pt 3333
7699717


In [3]:
import torch
vision_embedding = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/laion30m_caption/0.pt')
print(vision_embedding.shape)   

torch.Size([4096, 1024])


/tmp/ipykernel_580019/900397271.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vision_embedding = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/tensor_data

In [ ]:
import os
import glob
import torch
from tqdm import tqdm
file_dir = "/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-base/dreamclipcc12mhf"
files = glob.glob(os.path.join(file_dir, "*.pt"))
total_len = 0
for file in tqdm(files):
    tensor = torch.load(file, map_location=torch.device('cpu'), weights_only=False)
    total_len += tensor.size(0)
    if tensor.size(0) != 1024:
        print(file, tensor.size(0))
print(total_len)



In [ ]:
import os
import glob
import torch
from tqdm import tqdm
file_dir = "/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-giant/dreamclipcc3m"
files = glob.glob(os.path.join(file_dir, "*.pt"))
total_len = 0
for file in tqdm(files):
    tensor = torch.load(file, map_location=torch.device('cpu'), weights_only=False)
    total_len += tensor.size(0)
    if tensor.size(0) != 1024:
        print(file, tensor.size(0))
print(total_len)



In [ ]:
import pandas as pd
from data_config import DATADIR
# load csv file
csv = pd.read_csv(DATADIR['dreamclipcc12mhf']['annotation'], nrows=100000)
csv

In [ ]:
# open image and show
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
# 指定index为5的行是我们要显示的图像
index = random.randint(0, 100000)
image_path = os.path.join(DATADIR['dreamclipcc12mhf']['imagedir'], csv.iloc[index]['Image Path'])

# 打开图像并显示 标题为caption
image = Image.open(image_path)
plt.imshow(image)
plt.title("\n".join(csv.iloc[index]['longLLA_captions'].split(" ")), loc='left')
plt.show()


# 3: test if embedding are encoded correctly

In [1]:
import pandas as pd
import random
import os
from data_config import DATADIR
# csv = pd.read_csv('/home/mila/l/le.zhang/scratch/datasets/LAION/30M_laion_synthetic_filtered_large_with_path_filtered.csv')
# csv = pd.read_csv(DATADIR['laion30m']['annotation'])
csv = pd.read_csv(DATADIR['dreamclipcc12mhf']['annotation'])
csv

,Image Url,raw_caption,shortIB_captions,longIB_captions,shortSV_captions,longSV_captions,shortLLA_captions,longLLA_captions,Image Path
0,http://1.bp.blogspot.com/_dWmIOlGB7_0/S_HA9F1R...,this watch rolex gmt master has an independent...,a rolex watch with red and black dials,The image showcases a stunning silver and red ...,A Rolex watch with a black face and red and wh...,The image showcases a Rolex Submariner wristwa...,A silver and black watch with a red second hand.,The image features a silver and black wristwat...,CC12M_HF/images/0000000/0000000.jpg
1,http://2.bp.blogspot.com/-TwtZYTU4e9I/UQmEywkl...,picture of white modern bathroom furniture as ...,a white toilet sitting next to a sink in a bat...,The image showcases a modern and stylish bathr...,"A bathroom with a white toilet, sink, and mirror.","The image captures a modern bathroom, bathed i...",A bathroom with a toilet and sink.,The image depicts a modern bathroom with a whi...,CC12M_HF/images/0000000/0000002.jpg
2,http://2.bp.blogspot.com/--K8LdRsj99U/VQaOjr33...,the american cowboy chronicles native american...,a close up of a brown bear with its mouth open,The image features a large brown bear with its...,A close up of a bear with its mouth wide open.,The image captures a close-up view of a brown ...,A close up of a brown bear with its mouth open.,The image features a large brown bear with its...,CC12M_HF/images/0000000/0000003.jpg
3,http://2.bp.blogspot.com/-MBHbOy37tvU/TaX19Rrr...,assimilation process in the indian style and f...,a woman in a pink sari sitting on a bed,"In the picture, there are visible objects such...",A woman in a long pink dress sitting on a bed.,"In the image, a woman is the central figure, s...",A woman is sitting on a bed with a pink dress.,The image features a beautiful woman wearing a...,CC12M_HF/images/0000000/0000004.jpg
4,http://2.bp.blogspot.com/-2HQcTDQ4oQs/V1oBoudL...,in mourning afterglow album lyrics in mourning...,a painting of a lighthouse in the middle of a ...,The painting depicts a lighthouse standing on ...,A painting of a rocky island in the middle of ...,The image presents a dramatic scene of a rocky...,A painting of a lighthouse on a rocky island s...,The image depicts a dramatic scene of a lighth...,CC12M_HF/images/0000000/0000005.jpg
...,...,...,...,...,...,...,...,...,...
7699712,https://www.wtravelmagazine.com/wp-content/upl...,person on a yacht in cannes,a man and woman standing on a boat in the water,"In the image, there is a man and a woman posin...",person on a yacht in cannes,"In the image, a man and a woman are standing o...",A man and a woman are standing on a boat.,The image features a man and a woman standing ...,CC12M_HF/images/0001021/0005648.jpg
7699713,https://www.writeups.org/wp-content/uploads/Or...,orko he man and the masters of the universe ca...,two cartoon characters wearing hats and holdin...,"In the picture, there are two cartoon characte...",orko he man and the masters of the universe ca...,"In the image, we see a scene from the animated...",Two cartoon characters in witch costumes are s...,"The image features two cartoon characters, bot...",CC12M_HF/images/0001021/0005649.jpg
7699714,https://www.wubbanub.com/wp-content/uploads/ca...,old school ways to tell if youre having a boy ...,a boy and a girl holding hands walking down th...,"The image captures two young children, a boy a...",old school ways to tell if youre having a boy ...,"In the image, two children are seen walking do...",A boy and a girl are walking down a sidewalk h...,The image features a young boy and a girl walk...,CC12M_HF/images/0001021/0005656.jpg
7699715,https://www.wtravelmagazine.com/wp-content/upl...,ares station built purposefully within a large...,a group of people climbing through a cave at n...,"The image captures a group of three people, tw...",ares station built purposefully within a large...,The image captures a moment of exploration in ...,Two people are walking in a cave with their li...,The image depicts a group 

### 3.1 compare text local embedding and online embedding

In [2]:
import torch
import random

import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

model_path = 'Alibaba-NLP/gte-large-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
text_model = AutoModel.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.float16).to('cuda')
text_model.eval()

/home/mila/l/le.zhang/.conda/envs/openflamingo/lib/python3.9/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/mila/l/le.zhang/.conda/envs/openflamingo/lib/python3.9/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


NewModel(
  (embeddings): NewEmbeddings(
    (word_embeddings): Embedding(30528, 1024, padding_idx=0)
    (rotary_emb): NTKScalingRotaryEmbedding()
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): NewEncoder(
    (layer): ModuleList(
      (0-23): 24 x NewLayer(
        (attention): NewSdpaAttention(
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (mlp): NewGatedMLP(
          (up_gate_proj): Linear(in_features=1024, out_features=8192, bias=False)
          (down_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (act_fn): GELUActivation()
          (hidden_dropout): Dropout(p=0.1, inplace=False)
        )
        (attn_ln): LayerNorm((1024,), eps=1e-12, elementwis

In [24]:
# Requires transformers>=4.36.0

import torch.nn.functional as F
import random

def compare_embeddings(n_tests=20, verbose=False):
    def compute_difference(precomputed, computed):
        return F.mse_loss(precomputed, computed).item()

    differences = []
    all_test_index = []
    dimension = 512
    for _ in range(n_tests):
        file_idx = random.randint(0, 15038)
        file_idx = random.randint(0, 100)
        idx = random.randint(0, dimension-1)

        # Load pre-computed embeddings
        try:
            x = torch.load(f'/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/dreamclipcc12mhf_shortSV_captions/{file_idx}.pt', weights_only=True)
        except:
            continue

        # Get the caption and its pre-computed embedding
        caption = csv.iloc[file_idx*dimension+idx]['shortSV_captions']
        precomputed_embedding = x[idx]

        # Compute the embedding on-the-fly for comparison
        with torch.no_grad():
            inputs = tokenizer(caption, max_length=1024, padding=True, truncation=True, return_tensors='pt').to('cuda')
            outputs = text_model(**inputs)
            computed_embedding = outputs.last_hidden_state[:, 0].to(torch.float16)[0].cpu()

        # Compute the difference
        diff = compute_difference(precomputed_embedding, computed_embedding)
        if diff>0.5:
            print(f"remove {file_idx}.pt, with diff {diff}")
            # remove
            os.remove(f'/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/dreamclipcc12mhf_shortSV_captions/{file_idx}.pt')
        differences.append(diff)
        all_test_index.append(file_idx*dimension+idx)
        if verbose:
            print(f"Test {_ + 1}:")
            print(f"Caption: {caption} from index {file_idx*dimension+idx}")
            print(f"Precomputed embedding : {precomputed_embedding}")
            print(f"Computed embedding : {computed_embedding}")
            print(f"Difference (MSE): {diff}\n")

    # Print summary statistics
    print(f"All test index: {all_test_index}")
    print(f"Average difference over {n_tests} tests: {sum(differences) / n_tests}")
    print(f"Max difference: {max(differences)}")
    print(f"Min difference: {min(differences)}")

# Usage example:
compare_embeddings(n_tests=1000, verbose=False)


All test index: [50824, 25799, 35055, 38208, 47015, 44581, 23830, 46923, 33966, 47656, 26871, 14638, 19949, 16046, 20038, 28226, 18838, 18273, 44978, 16335, 22804, 351, 23500, 39967, 40409, 25521, 33986, 51091, 17851, 21771, 37924, 38174, 18518, 39327, 43927, 24986, 27933, 15199, 14810, 11559, 40069, 48833, 15341, 21771, 41813, 25910, 28034, 23124, 9909, 51525, 12739, 40886, 35941, 12306, 10337, 28606, 37064, 49656, 46351, 13619, 31399, 49750, 47890, 14840, 46247, 47372, 42798, 30550, 15482, 35453, 44096, 49182, 18756, 30411, 11678, 17913, 22241, 51673, 31962, 47547, 38070, 44589, 15430, 37341, 30028, 19721, 14049, 12579, 32082, 23190, 40090, 22356, 23764, 50970, 24690, 41450, 28412, 33790, 33633, 15848, 44217, 47546, 26774, 19856, 20712, 19248, 25544, 48129, 31303, 31568, 39630, 25925, 42872, 36084, 18808, 25534, 22982, 12709, 37380, 24401, 24804, 51377, 27816, 42238, 46898, 48963, 29967, 14291, 16920, 42926, 19202, 47744, 41400, 22791, 22364, 16141, 12855, 36365, 37183, 9873, 26548, 

: 

### 3.2 Test local image embedding and online computed embedding

In [2]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
import os
import random
import torch.nn.functional as F
import time

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-large')
model = AutoModel.from_pretrained('facebook/dinov2-large',attn_implementation="sdpa", torch_dtype=torch.float16)
# model = AutoModel.from_pretrained('facebook/dinov2-large')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# model = torch.compile(model)
model.eval()

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-23): 24 x Dinov2Layer(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2SdpaAttention(
          (attention): Dinov2SdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
      

In [5]:
def compare_image_embeddings(model,processor,n_tests=5, verbose=False):
    differences = []
    all_test_indices = []
    device = model.device

    for idx in range(800):
        # file_idx = random.randint(0, 5169)
        file_idx = idx
        idx = random.randint(0, 4096)  # Changed to 0-4095 to match the tensor size
        test_index = file_idx * 4096 + idx
        all_test_indices.append(test_index)

        # image_path = os.path.join(DATADIR['laion30m']['imagedir'], csv.iloc[test_index]['Image Path'])
        image_path = os.path.join(DATADIR['dreamclipcc12mhf']['imagedir'], csv.iloc[test_index]['Image Path'])
        image = Image.open(image_path)
        
        
        if verbose:
            print(f"Test {_ + 1}:")
            print(f"Comparing image at index {test_index}")

        # Load pre-computed embedding
        # emebdding_path = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m/{file_idx}.pt"
        emebdding_path = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/dreamclipcc12mhf/{file_idx}.pt"
        if not os.path.exists(emebdding_path):
            continue
        x = torch.load(emebdding_path, weights_only=True)
        precomputed_embedding = x[idx].to(device)

        # Compute embedding on-the-fly
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        sequence_output = outputs[0]
        cls_token = sequence_output[:, 0]
        patch_tokens = sequence_output[:, 1:]
        computed_embedding = torch.cat([cls_token, patch_tokens.mean(dim=1)], dim=1).to(torch.float16)[0]

        # Move embeddings back to CPU for comparison and storage
        precomputed_embedding = precomputed_embedding.cpu()
        computed_embedding = computed_embedding.cpu()

        if verbose:
            print(f"Precomputed embedding: {precomputed_embedding}")
            print(f"Computed embedding: {computed_embedding}")

        # Compute the difference
        diff = F.mse_loss(precomputed_embedding, computed_embedding).item()
        differences.append(diff)
        if diff>1:
            # os.remove(emebdding_path)
            print(f"remove {emebdding_path}, with diff {diff}")
            
        if verbose:
            print(f"Difference (MSE): {diff}\n")

    # Print summary statistics
    print(f"All test indices: {all_test_indices}")
    print(f"Average difference over {n_tests} tests: {sum(differences) / n_tests}")
    print(f"Max difference: {max(differences)}")
    print(f"Min difference: {min(differences)}")

start_time = time.time()
# Usage example:
compare_image_embeddings(model,processor,n_tests=1000, verbose=False)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

All test indices: [758, 6989, 8491, 16173, 17461, 23372, 25892, 32297, 36039, 39138, 41149, 46913, 53235, 54677, 60544, 65380, 67525, 72672, 76540, 78548, 83515, 88005, 93093, 98146, 100380, 104508, 106756, 114029, 117111, 122433, 124935, 129129, 134129, 136543, 143182, 143660, 147480, 154354, 159021, 162603, 167728, 170706, 173056, 177564, 183608, 184972, 188936, 193194, 197755, 202909, 208269, 212327, 216781, 219585, 223301, 225692, 231168, 233950, 238622, 242849, 247468, 253450, 254373, 258695, 265637, 270327, 274335, 277548, 281753, 285543, 288175, 291804, 298724, 302602, 304492, 307337, 311954, 317898, 322158, 326072, 329540, 333895, 336904, 343121, 345375, 350332, 355900, 356968, 361502, 365375, 369597, 375128, 377914, 384876, 386960, 392357, 393303, 401057, 404012, 409352, 411668, 416957, 419407, 422562, 427804, 432909, 438219, 441406, 445331, 448507, 450943, 456582, 459755, 466002, 468486, 475071, 475252, 481481, 483746, 489823, 491606, 497783, 502102, 504202, 509822, 512273, 5

### 3.2.1 Manually inspect single file

In [111]:
from PIL import Image
import torch

device = 'cuda'
file_idx = random.randint(0, 5000)
idx = random.randint(0, 4096)  # Changed to 0-4095 to match the tensor size
test_index = file_idx * 4096 + idx

image_path = os.path.join(DATADIR['laion30m']['imagedir'], csv.iloc[test_index]['Image Path'])
image = Image.open(image_path)


# Load pre-computed embedding
emebdding_path = f"/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m/{file_idx}.pt"

x = torch.load(emebdding_path, weights_only=True)
precomputed_embedding = x[idx].to(device)

# Compute embedding on-the-fly
inputs = processor(images=image, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)
sequence_output = outputs[0]
cls_token = sequence_output[:, 0]
patch_tokens = sequence_output[:, 1:]
computed_embedding = torch.cat([cls_token, patch_tokens.mean(dim=1)], dim=1).to(torch.float16)[0]

# Move embeddings back to CPU for comparison and storage
precomputed_embedding = precomputed_embedding.cpu()
computed_embedding = computed_embedding.cpu()
print(f"image index {test_index}")
print(f"Precomputed embedding: {precomputed_embedding}")
print(f"Computed embedding: {computed_embedding}")

FileNotFoundError: [Errno 2] No such file or directory: '/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/image_embedding/dinov2-large/laion30m/1272.pt'

### 3.3 Visualize the image and its caption

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random
# Get the caption and image path
idx = random.randint(0, len(df))
caption = df.iloc[idx]['caption']
image_path = os.path.join("/home/mila/l/le.zhang/scratch/datasets/LAION", df.iloc[idx]['Image Path'])

# Open and display the image
img = Image.open(image_path)
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')

# Add the caption at the bottom of the image
plt.text(0.5, -0.05, caption, wrap=True, horizontalalignment='center', 
         verticalalignment='top', transform=plt.gca().transAxes)

plt.show()



In [4]:
import torch 
import random
x1 = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embedding/gte-large-en-v1.5/validation/406.pt')
x1.shape

/tmp/ipykernel_2501911/788722818.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x1 = torch.load('/home/mila/l/le.zhang/scratch/light_align/data/tensor_data/text_embeddi

torch.Size([4096, 1024])